# **LSTM for Part-of-Speech Tagging**
Part of speech tagging is the process of determining the category of a word from the words in its surrounding context.

**Why Tag Speech?**
* Often used to help disambiguate natural language phrases because it can be done quickly and with high accuracy. 
* It can help answer: what subject is someone talking about?
* Tagging can be used for many NLP tasks like creating new sentences using a sequence of tags that make sense together, filling in a Mad Libs style game, and determining correct pronunciation during speech synthesis. 
* It is also used in information retrieval, and for word disambiguation (ex. determining when someone says right like the direction versus right like "that's right!").

# **Preparing the Data**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# training sentences and their corresponding word-tags
# DET: Determinant, NN: Noun, V: Verb
training_data = [
  ("The cat ate the cheese".lower().split(), ["DET", "NN", "V", "DET", "NN"]),
  ("She read that book".lower().split(), ["NN", "V", "DET", "NN"]),
  ("The dog loves art".lower().split(), ["DET", "NN", "V", "NN"]),
  ("The elephant answers the phone".lower().split(), ["DET", "NN", "V", "DET", "NN"])
]

print("training_data:: ", training_data)

training_data::  [(['the', 'cat', 'ate', 'the', 'cheese'], ['DET', 'NN', 'V', 'DET', 'NN']), (['she', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN']), (['the', 'dog', 'loves', 'art'], ['DET', 'NN', 'V', 'NN']), (['the', 'elephant', 'answers', 'the', 'phone'], ['DET', 'NN', 'V', 'DET', 'NN'])]


In [ ]:
# create a dictionary that maps words to indices
word2idx = {}
for sent, tags in training_data:
  for word in sent:
    if word not in word2idx:
      word2idx[word] = len(word2idx)

print("word2idx:: ", word2idx)

# create a dictionary that maps tags to indices
tag2idx = {"DET": 0, "NN": 1, "V": 2}

word2idx::  {'the': 0, 'cat': 1, 'ate': 2, 'cheese': 3, 'she': 4, 'read': 5, 'that': 6, 'book': 7, 'dog': 8, 'loves': 9, 'art': 10, 'elephant': 11, 'answers': 12, 'phone': 13}


In [ ]:
import numpy as np
# a helper function for converting a sequence of words to a Tensor of numerical values
# will be used later in training

def prepare_sequence(seq, to_idx):
  '''This function takes in a sequence of words and returns a 
    corresponding Tensor of numerical values (indices for each word).'''
  idxs = [to_idx[w] for w in seq]
  idxs = np.array(idxs)
  return torch.from_numpy(idxs)

In [ ]:
example_input = prepare_sequence("The dog answers the phone".lower().split(), word2idx)
print("example_input", example_input)

example_input tensor([ 0,  8, 12,  0, 13])


---

# **Creating the Model**
Our model will assume a few things:
1. Our input is broken down into a sequence of words, so a sentence will be [w1, w2, ...]
2. These words come from a larger list of words that we already know (a vocabulary)
3. We have a limited set of tags, [NN, V, DET], which mean: a noun, a verb, and a determinant (words like "the" or "that"), respectively
4. We want to predict a tag for each input word

To do the prediction, we will pass an LSTM over a test sentence and apply a softmax function to the hidden state of the LSTM; the result is a vector of tag scores from which we can get the predicted tag for a word based on the maximum value in this distribution of tag scores.

**Word embeddings**

LSTM takes in an expected input size and hidden_dim, but sentences are rarely of a consistent size.

At the very start of this net, we'll create an Embedding layer that takes in the size of our vocabulary and returns a vector of a specified size, embedding_dim, for each word in an input sequence of words. 

It's important that this be the first layer in this net. 

In [10]:
class LSTMTagger(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
    super(LSTMTagger,self).__init__()

    self.hidden_dim = hidden_dim

    # embedding layer that turns words into a vector of a specified size
    self.word_embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

    # the LSTM takes embedded word vectors (of a specified size) as inputs 
      # and outputs hidden states of size hidden_dim
    self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim)

    # the linear layer that maps the hidden state output dimension 
      # to the number of tags we want as output, tagset_size (in this case this is 3 tags)
    self.hidden2tag = nn.Linear(in_features=hidden_dim, out_features=tagset_size)

    # initialize the hidden state
    self.hidden = self.init_hidden()

  def init_hidden(self):
    ''' At the start of training, we need to initialize a hidden state;
        there will be none because the hidden state is formed based on perviously seen data.
        So, this function defines a hidden state with all zeroes and of a specified size.'''
    # The axes dimensions are (n_layers, batch_size, hidden_dim)
    return (torch.zeros(1, 1, self.hidden_dim), torch.zeros(1, 1, self.hidden_dim))

  def forward(self, sentence):
    # create embedded word vectors for each word in a sentence
    embeds = self.word_embeddings(sentence)

    # get the output and hidden state by passing the lstm over our word embeddings
      # the lstm takes in our embeddings and hiddent state
    lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)

    # get the scores for the most likely tag for a word
    tag_outputs = self.hidden2tag(lstm_out.view(len(sentence), -1))
    tag_scores = F.log_softmax(input=tag_outputs, dim=1)

    return tag_scores


In [11]:
# the embedding dimension defines the size of our word vectors
# for our simple vocabulary and training set, we will keep these small

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# instantiate our model
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), len(tag2idx))

# define loss and optimizer
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [12]:
test_sentence = "The cheese loves the elephant".lower().split()

# see what the scores are before training
# element [i,j] of the output is the *score* for tag j for word i.
# to check the initial accuracy of our model, we don't need to train, so we use model.eval()
inputs = prepare_sequence(test_sentence, word2idx)
inputs = inputs
tag_scores = model(inputs)
print("tag_scores", tag_scores)

# tag_scores outputs a vector of tag scores for each word in an inpit sentence
# to get the most likely tag index, we grab the index with the maximum score!
# recall that these numbers correspond to tag2idx = {"DET": 0, "NN": 1, "V": 2}
_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tag_scores tensor([[-1.4876, -1.1221, -0.8019],
        [-1.6764, -1.1624, -0.6927],
        [-1.5466, -1.1654, -0.7440],
        [-1.5359, -1.1410, -0.7652],
        [-1.5345, -1.0699, -0.8178]], grad_fn=<LogSoftmaxBackward>)


Predicted tags: 
 tensor([2, 2, 2, 2, 2])


In [13]:
# normally these epochs take a lot longer 
# but with our toy data (only 3 sentences), we can do many epochs in a short time
n_epochs = 300

for epoch in range(n_epochs):
    
    epoch_loss = 0.0
    
    # get all sentences and corresponding tags in the training data
    for sentence, tags in training_data:
        
        # zero the gradients
        model.zero_grad()

        # zero the hidden state of the LSTM, this detaches it from its history
        model.hidden = model.init_hidden()

        # prepare the inputs for processing by out network, 
        # turn all sentences and targets into Tensors of numerical indices
        sentence_in = prepare_sequence(sentence, word2idx)
        targets = prepare_sequence(tags, tag2idx)

        # forward pass to get tag scores
        tag_scores = model(sentence_in)

        # compute the loss, and gradients 
        loss = loss_function(tag_scores, targets)
        epoch_loss += loss.item()
        loss.backward()
        
        # update the model parameters with optimizer.step()
        optimizer.step()
        
    # print out avg loss per 20 epochs
    if(epoch%20 == 19):
        print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(training_data)))

Epoch: 20, loss: 1.10391
Epoch: 40, loss: 1.06142
Epoch: 60, loss: 1.03421
Epoch: 80, loss: 1.01339
Epoch: 100, loss: 0.99488
Epoch: 120, loss: 0.97664
Epoch: 140, loss: 0.95764
Epoch: 160, loss: 0.93733
Epoch: 180, loss: 0.91548
Epoch: 200, loss: 0.89204
Epoch: 220, loss: 0.86711
Epoch: 240, loss: 0.84087
Epoch: 260, loss: 0.81356
Epoch: 280, loss: 0.78547
Epoch: 300, loss: 0.75686


In [14]:
test_sentence = "The cheese loves the elephant".lower().split()

# see what the scores are after training
inputs = prepare_sequence(test_sentence, word2idx)
inputs = inputs
tag_scores = model(inputs)
print(tag_scores)

# print the most likely tag index, by grabbing the index with the maximum score!
# recall that these numbers correspond to tag2idx = {"DET": 0, "NN": 1, "V": 2}
_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-0.5204, -1.2120, -2.2246],
        [-1.5089, -0.4769, -1.8444],
        [-1.5630, -0.6654, -1.2859],
        [-0.6430, -1.1018, -1.9518],
        [-1.6399, -0.5664, -1.4336]], grad_fn=<LogSoftmaxBackward>)


Predicted tags: 
 tensor([0, 1, 1, 0, 1])
